#### **The NN Module**
The torch.nn module in PyTorch is a core library that provides a wide array of classes and functions designed to help developers build neural networks efficently and effectively. It abstracts the complexity of creating and training neural networks by offering pre-build layers, loss functions, activation functions and other utilities, enabling you to focus on designing and experimenting with model architectures

# Key Components of torch.nn :

1. Modules (Layers) :
  - nn.Module: The base class for all neural netword modules. Your custom models and layers should subclass this class.
  - Common Layers : Includes layers like nn.Linear (fully connected layers), nn.Conv2d (convolutional layer), nn.LSTM (recurrent layer), and many more.

2. Activation Functions :
 - Functions like nn.RelU, nn.Sigmoid, nn.Tanh introduce non-linearities to the model, allowing it to learn complex patterns

3. Loss Functions :
  - Provides loss functions such as nn.CrossEntropyLoss, nn.MSELoss and nn.NLLLoss to quantify the difference between the model's prediction and the actual targets

4. Contianer Modules :
  - nn.Sequentail : A sequential container to stack layers in order.

5. Regularization and Dropout :     
  - Layers like nn.Dropout and nn.BatchNorm2d help prevent overfitting and improve the model's ability to generalize to new data.

#### Creating a simple model with 1 perceptron

In [6]:
# Creating the model
import torch
import torch.nn as nn

class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

    # Forward Pass
  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)

    return out


In [7]:
# Creating Dataset
features = torch.rand(10, 5)

# Create model
model = Model(features.shape[1])

# Call model for forward pass
model(features)

tensor([[0.6199],
        [0.5761],
        [0.6846],
        [0.5583],
        [0.6806],
        [0.5842],
        [0.5914],
        [0.5590],
        [0.6063],
        [0.6324]], grad_fn=<SigmoidBackward0>)

In [8]:
# show model weights
model.linear.weight

Parameter containing:
tensor([[ 0.1116,  0.3117,  0.4396, -0.1351, -0.0379]], requires_grad=True)

In [11]:
# bises
model.linear.bias

Parameter containing:
tensor([0.0395], requires_grad=True)

In [12]:
!pip install torchinfo

In [13]:
from torchinfo import summary

summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

#### Creating a Model with hidden layers

In [22]:
import torch
import torch.nn as nn

# Defining the Model

class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear1 = nn.Linear(num_features, 3)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(3, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear1(features)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)
    return out

In [23]:
# Creating the Dataset
features = torch.rand(10,5)

# Creating the Model
model = Model(features.shape[1])

# Call model for forward pass
model(features)

tensor([[0.6880],
        [0.6956],
        [0.6732],
        [0.7021],
        [0.7210],
        [0.7056],
        [0.6976],
        [0.6796],
        [0.7106],
        [0.7044]], grad_fn=<SigmoidBackward0>)

In [24]:
model.linear1.weight

Parameter containing:
tensor([[ 0.2507,  0.0648,  0.1973, -0.0064,  0.0322],
        [ 0.3956,  0.1902, -0.0590,  0.1214,  0.2612],
        [ 0.3122, -0.1554, -0.0668,  0.2562,  0.3379]], requires_grad=True)

In [25]:
model.linear2.weight

Parameter containing:
tensor([[0.4146, 0.0334, 0.3849]], requires_grad=True)

In [27]:
model.linear1.bias

Parameter containing:
tensor([-0.1210,  0.1035,  0.3551], requires_grad=True)

In [28]:
model.linear2.bias

Parameter containing:
tensor([0.4602], requires_grad=True)

In [29]:
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

#### Defining model using Sequential Containers

In [30]:
import torch
import torch.nn as nn

# Defining the Model

class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features, 3),
        nn.ReLU(),
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)
    return out

In [31]:
# Creating the Dataset
features = torch.rand(10,5)

# Creating the Model
model = Model(features.shape[1])

# Call model for forward pass
model(features)

tensor([[0.4445],
        [0.4422],
        [0.4188],
        [0.4391],
        [0.4344],
        [0.4398],
        [0.4277],
        [0.4404],
        [0.4284],
        [0.4341]], grad_fn=<SigmoidBackward0>)

#### Using nn.Module for Breast Cancer

##### Imports

In [41]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [42]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")

##### Dataset Preprocessing

In [43]:
df.head()
df.drop(columns=["Unnamed: 32","id"],inplace=True)

##### Splitting, Scaling and Encoding

In [44]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

##### Converting into PyTorch Tensor

In [61]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

##### Defining the Model

In [62]:
class MySimpleNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features, 1),
        nn.Sigmoid()
    )
  def forward(self, features):
    return self.network(features)

  def loss_function(self, y_pred, y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    loss = - (y * torch.log(y_pred) + (1-y) * torch.log(1 - y_pred)).mean()
    return loss

In [63]:
learning_rate = 0.1
epochs = 25

In [68]:
model = MySimpleNN(X_train_tensor.shape[1])

for epoch in range(epochs):
  y_pred = model(X_train_tensor)
  loss = model.loss_function(y_pred, y_train_tensor)
  loss.backward()

  with torch.no_grad():
    model.network[0].weight -= learning_rate * model.network[0].weight.grad
    model.network[0].bias -= learning_rate * model.network[0].bias.grad

    model.network[0].weight.grad.zero_()
    model.network[0].bias.grad.zero_()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.7343925833702087
Epoch 2, Loss: 0.7121169567108154
Epoch 3, Loss: 0.6985650062561035
Epoch 4, Loss: 0.690415620803833
Epoch 5, Loss: 0.685405433177948
Epoch 6, Loss: 0.6821612119674683
Epoch 7, Loss: 0.6799104809761047
Epoch 8, Loss: 0.6782352924346924
Epoch 9, Loss: 0.6769103407859802
Epoch 10, Loss: 0.6758124232292175
Epoch 11, Loss: 0.6748707890510559
Epoch 12, Loss: 0.6740435361862183
Epoch 13, Loss: 0.6733037829399109
Epoch 14, Loss: 0.6726335287094116
Epoch 15, Loss: 0.6720203161239624
Epoch 16, Loss: 0.671454906463623
Epoch 17, Loss: 0.6709303855895996
Epoch 18, Loss: 0.6704415082931519
Epoch 19, Loss: 0.6699838638305664
Epoch 20, Loss: 0.6695542335510254
Epoch 21, Loss: 0.6691496968269348
Epoch 22, Loss: 0.6687681078910828
Epoch 23, Loss: 0.6684073209762573
Epoch 24, Loss: 0.6680659055709839
Epoch 25, Loss: 0.6677421927452087


In [69]:
# Model Evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.6184980273246765


##### Using Built-in Loss function and Optimizer

**The torch.optim module**

***torch.optim*** is a module in PyTorch that provides a variety of optimization algorithms used to update the parameters of your model during training

It includes common optimizers like Dtochastic Gradient Descent(SGD), Adam, RMSprop and more.

It handles weight updates efficiently, including additional features like learning rate scheduling and weight decay (regularization).

The **model.parameters()** method in PyTorch retrieves an **iterator over all the trainable parameters (weights and bias)**  in a model. These parameters are instances of torch.nn.Parameter and include :     
- Weights : The weight metrices of layers like nn.Linear, nn.Conv2d, etc
- Biases : The bias term of layers (if they exist)

The optimizer uses these parameters to compute gradients and update them during training

In [70]:
loss_function = nn.BCELoss()

In [74]:
model = MySimpleNN(X_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  y_pred = model(X_train_tensor)
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1)) # View is similar to reshape but more efficient for continuous data

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.6816350817680359
Epoch 2, Loss: 0.5194056630134583
Epoch 3, Loss: 0.43324726819992065
Epoch 4, Loss: 0.38031938672065735
Epoch 5, Loss: 0.34405866265296936
Epoch 6, Loss: 0.31734389066696167
Epoch 7, Loss: 0.2966401278972626
Epoch 8, Loss: 0.27999168634414673
Epoch 9, Loss: 0.2662251889705658
Epoch 10, Loss: 0.2545914053916931
Epoch 11, Loss: 0.24458767473697662
Epoch 12, Loss: 0.23586316406726837
Epoch 13, Loss: 0.22816474735736847
Epoch 14, Loss: 0.22130464017391205
Epoch 15, Loss: 0.2151402086019516
Epoch 16, Loss: 0.20956094563007355
Epoch 17, Loss: 0.20447967946529388
Epoch 18, Loss: 0.19982647895812988
Epoch 19, Loss: 0.19554461538791656
Epoch 20, Loss: 0.1915873885154724
Epoch 21, Loss: 0.18791599571704865
Epoch 22, Loss: 0.1844978928565979
Epoch 23, Loss: 0.1813054382801056
Epoch 24, Loss: 0.1783151626586914
Epoch 25, Loss: 0.17550675570964813


In [73]:
# Model Evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.5301631093025208
